# Tacotron-2 training

This notebook provides examples to 
1. Create a new Tacotron2 model (pretrained or new one with transfer-learning) \*
2. Load dataset, test and analyze it
3. Train the model on this dataset
4. Complete inference with the `Waveglow` NVIDIA's pretrained vocoder

\* **Important** : when creating a new model with transfer learning (or from pretrained pytorch model), both models (pretrained and new one) are loaded in memory. It is necessary to restart the kernel before training to avoid `Out Of Memory (OOM)` errors !

Note : this notebook does **not** retrain a model till convergence because some powerful French models are already shared. The execution is simply to assess that the code is working, and to give an example of outputs.

The complete training procedure to have good performance model takes around 15 epochs (at least on SIWIS) with the `NVIDIA's pretrained` model (for `transfer-learning`), which takes (on my GeForce GTX1070) around 15h. 
It means that, in approximately 1 night of training, you can have a really good `Text-To-Speech synthesis` : quite impressive and funny !

**Important Note** : if you do not have a working `pytorch GPU` installation, you have to download the `WaveGlow` weights for tensorflow, as well as the `pretrained_tacotron2` model (cf `README` file).

## Imports + model creation

The 1st cell imports all necessary functions and define the global variable `model_name` which is the name of the model used in the whole notebook. 

The 4 next cells create new `Tacotron2` in different ways.

In [1]:
import pandas as pd
import tensorflow as tf

from models import get_pretrained
from models.tts import Tacotron2, WaveGlow
from custom_architectures import get_architecture
from datasets import get_dataset, train_test_split, prepare_dataset, test_dataset_time
from utils import plot_spectrogram, limit_gpu_memory
from utils.text import default_french_encoder
from utils.audio import display_audio, load_audio, load_mel

gpus = tf.config.list_physical_devices('GPU')

rate = 22050
model_name = "tacotron2_siwis"

print("Tensorflow version : {}".format(tf.__version__))
print("Available GPU's ({}) : {}".format(len(gpus), gpus))

Tensorflow version : 2.10.0
Available GPU's (1) : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# This cell creates a perfect copy of the NVIDIA's pretrained model (and name it "nvidia_pretrained")
model = Tacotron2.from_nvidia_pretrained()
print(model)

In [2]:
# This special cleaner allow to not lowercase the text 
# see my data_processing repository for more examples on text encoding / cleaners
# If you want lowercase, you just have to remove the "cleaners" argument from default_french_encoder()
cleaners = [
    {'name' : 'french_cleaners', 'to_lowercase' : False}
]
encoder = default_french_encoder(vocab_size = 148, cleaners = cleaners)
print(encoder)

========== Text encoder ==========
Vocab (size = 148) : ['_', '-', '!', "'", '(', ')', ',', '.', ':', ';', '?', ' ', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']
Config : {
  "level": 0,
  "lstrip": false,
  "rstrip": false,
  "cleaners": [
    {
      "name": "french_cleaners",
      "to_lowercase": false
    }
  ],
  "split_pattern": null,
  "bpe_end_of_word": null,
  "pad_token": "",
  "sep_token": null,
  "ukn_token": null,
  "sos_token": "[SOS]",
  "eos_token": "[EOS]",
  "mask_token": null,
  "sub_word_prefix": "",
  "use_sos_and_eos": false
}


In [ ]:
# This cell creates a new model based on the NVIDIA's pretrained model
model = Tacotron2.from_nvidia_pretrained(
    nom = model_name, lang = "fr", text_encoder = encoder
)
print(model)

In [4]:
# This cell creates a new model based on a pretrained Tacotron2 model
model = Tacotron2.from_pretrained(
    nom = model_name, pretrained_name = 'pretrained_tacotron2',
    lang = "fr", text_encoder = encoder
)
print(model)

Model restoration...
Initializing submodel : `tts_model` !
Successfully restored tts_model from pretrained_models/pretrained_tacotron2/saving/tts_model.json !
Model pretrained_tacotron2 initialized successfully !
Initializing model with kwargs : {'tts_model': {'architecture_name': 'tacotron2', 'pad_token': 0, 'vocab_size': 148, 'n_mel_channels': 80, 'init_step': 0}}
Initializing submodel : `tts_model` !
Submodel tts_model saved in pretrained_models\tacotron2_siwis\saving\tts_model.json !
Model tacotron2_siwis initialized successfully !
Weights transfered successfully !
Submodel tts_model saved in pretrained_models\tacotron2_siwis\saving\tts_model.json !

========== tacotron2_siwis ==========
Sub model tts_model
- Inputs 	: unknown
- Outputs 	: unknown
- Number of layers 	: 3
- Number of parameters 	: 28.190 Millions
- Model not compiled

Transfer-learning from : pretrained_tacotron2
Already trained on 0 epochs (0 steps)

- Language : fr
- Vocabulary (size = 148) : ['_', '-', '!', "'", 

## Model instanciation

This cell loads the model based on its name. Once created, you **do not have** to put all its configuration again : they will be loaded automatically ! Furthermore, models are `singleton` so you can execute these cell as many times as you want but the model will not be reloaded every times !

In [2]:
model = get_pretrained(model_name)

model.compile(
    optimizer = 'adam', 
    optimizer_config = {
        'lr' : {
            'name' : 'WarmupScheduler',
            'maxval' : 1e-3,
            'minval' : 1e-4,
            'factor' : 128,
            'warmup_steps' : 512
        }
    }
)

print(model)

Model restoration...
Initializing submodel : `tts_model` !
Successfully restored tts_model from pretrained_models/tacotron2_siwis/saving/tts_model.json !
Model tacotron2_siwis initialized successfully !
Optimizer 'tts_model_optimizer' initilized successfully !
Submodel tts_model compiled !
  Loss : {'reduction': 'none', 'name': 'tacotron_loss', 'mel_loss': 'mse', 'mask_mel_padding': True, 'label_smoothing': 0, 'finish_weight': 1.0, 'not_finish_weight': 1.0, 'from_logits': False}
  Optimizer : {'name': 'Adam', 'learning_rate': {'class_name': 'WarmupScheduler', 'config': {'factor': 128.0, 'warmup_steps': 512, 'minval': 0.0001, 'maxval': 0.001}}, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
  Metrics : []

========== tacotron2_siwis ==========
Sub model tts_model
- Inputs 	: unknown
- Outputs 	: unknown
- Number of layers 	: 3
- Number of parameters 	: 28.190 Millions
- Optimizer 	: {'name': 'Adam', 'learning_rate': {'class_name': 'WarmupScheduler', 'c

In [3]:
dataset_name = 'siwis'
dataset = get_dataset(dataset_name)

train, valid = None, None

print("Dataset length : {}".format(len(dataset)))

Loading dataset siwis...
Dataset length : 9763


In [4]:
dataset.head()

,text,filename,mels_22050_chann-80_filt-1024_hop-256_win-1024_norm-None,wavs_16000,wavs_22050,wavs_44100,time,dataset_name,id
0,"Benoît Hamon, monsieur le ministre, ce texte, ...",D:/datasets/SIWIS\fr\wavs\part1\neut_parl_s01_...,D:/datasets/SIWIS\fr\mels_22050_chann-80_filt-...,D:/datasets/SIWIS\fr\wavs_16000\part1\neut_par...,D:/datasets/SIWIS\fr\wavs_22050\part1\neut_par...,D:/datasets/SIWIS\fr\wavs\part1\neut_parl_s01_...,-1.0,siwis,siwis
1,Cette lutte se situe à deux niveaux.,D:/datasets/SIWIS\fr\wavs\part1\neut_parl_s01_...,D:/datasets/SIWIS\fr\mels_22050_chann-80_filt-...,D:/datasets/SIWIS\fr\wavs_16000\part1\neut_par...,D:/datasets/SIWIS\fr\wavs_22050\part1\neut_par...,D:/datasets/SIWIS\fr\wavs\part1\neut_parl_s01_...,-1.0,siwis,siwis
2,Venons-en maintenant au fond.,D:/datasets/SIWIS\fr\wavs\part1\neut_parl_s01_...,D:/datasets/SIWIS\fr\mels_22050_chann-80_filt-...,D:/datasets/SIWIS\fr\wavs_16000\part1\neut_par...,D:/datasets/SIWIS\fr\wavs_22050\part1\neut_par...,D:/datasets/SIWIS\fr\wavs\part1\neut_parl_s01_...,-1.0,siwis,siwis
3,"Peu à peu, ils mobilisent des moyens.",D:/datasets/SIWIS\fr\wavs\part1\neut_parl_s01_...,D:/datasets/SIWIS\fr\mels_22050_chann-80_filt-...,D:/datasets/SIWIS\fr\wavs_16000\part1\neut_par...,D:/datasets/SIWIS\fr\wavs_22050\part1\neut_par...,D:/datasets/SIWIS\fr\wavs\part1\neut_parl_s01_...,-1.0,siwis,siwis
4,"S’il y a unanimité pour augmenter le volume, n...",D:/datasets/SIWIS\fr\wavs\part1\neut_parl_s01_...,D:/datasets/SIWIS\fr\mels_22050_chann-80_filt-...,D:/datasets/SIWIS\fr\wavs_16000\part1\neut_par...,D:/datasets/SIWIS\fr\wavs_22050\part1\neut_par...,D:/datasets/SIWIS\fr\wavs\part1\neut_parl_s01_...,-1.0,siwis,siwis


## Training

The model is quite easy to train and has many `training hyperparameters` for audio processing

You can easily train your own model on your dataset : the only information required by the model is a `pd.DataFrame` with `filename` and `text` columns (other fields are optional : the `wavs_22050` is the resampled file to speed up audio loading).

The `id` and `time` fields are optional (used for dataset analysis).

You can see `max_train_frames` and `pad_to_multiple` configuration but they are currently **not** supported. These are for splitted training where we train on the complete mel but by splitting it in sub parts but this is not working yet

The `SIWIS` dataset is a really good quality dataset so it does not need any trimming / processing. I left the parameters to facilitate their modification if necessary for other datasets

As you can observe, multiple losses are displayed (the general `loss` + `mel_loss`, `gate_loss` and `mel_postnet_loss`) : the `TacotronLoss` computes them to give a more *in-depth* view of the model performances. For the `wiehgts update`, only `loss` is used (which is the sum of the 3 other losses).

The `train_size` and `valid_size` parameters can be given to `model.train` and the split will be done internally. I have decided to make it before training to show the actual number of batches ;)

In [6]:
""" Classic hyperparameters """
epochs     = 5
batch_size = 32
valid_batch_size = 2 * batch_size
train_prop = 0.9
train_size = 640 #int(len(dataset) * train_prop)
valid_size = 640 #min(len(dataset) - train_size, 250 * valid_batch_size)

shuffle_size    = 16 * batch_size
pred_step       = -1 # make a prediction after every epoch
augment_prct    = 0.25

""" Custom training hparams """
trim_audio      = False
reduce_noise    = False
trim_threshold  = 0.075
max_silence     = 0.25
trim_method     = 'window'
trim_mode       = 'start_end'

trim_mel     = False
trim_factor  = 0.6
trim_mel_method  = 'max_start_end'

# These lengths corresponds to approximately 5s audio
# This is the max my GPU supports for an efficient training but is large enough for the SIWIS dataset
max_output_length = 512
max_input_length = 75

""" Training """

# this is to normalize dataset usage so that you can use a pre-splitted dataset or not
# without changing anything in the training configuration
if train is None or valid is None:
    train, valid = train_test_split(
        dataset, train_size = train_size, valid_size = valid_size, shuffle = True
    )

print("Training samples   : {} - {} batches".format(
    len(train), len(train) // batch_size
))
print("Validation samples : {} - {} batches".format(
    len(valid), len(valid) // valid_batch_size
))

model.train(
    train, validation_data = valid, 

    epochs = epochs, batch_size = batch_size, valid_batch_size = valid_batch_size,
    
    max_input_length = max_input_length, max_output_length = max_output_length,
    pred_step = pred_step, shuffle_size = shuffle_size, augment_prct = augment_prct,
    
    trim_audio = trim_audio, reduce_noise = reduce_noise, trim_threshold = trim_threshold,
    max_silence = max_silence, trim_method = trim_method, trim_mode = trim_mode,
    
    trim_mel = trim_mel, trim_factor = trim_factor, trim_mel_method = trim_mel_method,
)

Training samples   : 640 - 20 batches
Validation samples : 640 - 10 batches
Training config :
HParams :
- augment_prct	: 0.25
- trim_audio	: False
- reduce_noise	: False
- trim_threshold	: 0.075
- max_silence	: 0.25
- trim_method	: window
- trim_mode	: start_end
- trim_mel	: False
- trim_factor	: 0.6
- trim_mel_method	: max_start_end
- max_input_length	: 75
- max_output_length	: 512
- max_train_frames	: -1
- pad_to_multiple	: False
- batch_size	: 32
- train_batch_size	: None
- valid_batch_size	: 64
- test_batch_size	: 1
- shuffle_size	: 512
- epochs	: 5
- verbose	: 1
- train_times	: 1
- valid_times	: 1
- train_size	: None
- valid_size	: None
- test_size	: 4
- pred_step	: -1

Running on 1 GPU

Epoch 1 / 5
Epoch 1/5
     10/Unknown - 89s 7s/step - loss: 3.9265 - mse_mel_loss: 2.0723 - mse_mel_postnet_loss: 1.8121 - gate_loss: 0.0421Training interrupted ! Saving model...
Training interrupted at epoch 0 !
Training finished after 1min 35sec !
Submodel tts_model saved in pretrained_models\ta

In [ ]:
model.plot_history()

Just to show the power of the `History` configuration tracking :D

In [8]:
print("Training informations :")
print(pd.DataFrame(model.history.trainings_infos))
print("Training configurations :")
pd.DataFrame(model.history.trainings_config)

Training informations :
                       start                        end       time   
0 2023-04-29 15:40:33.606389 2023-04-29 15:42:09.077067  95.470678  \

   interrupted  start_epoch  final_epoch  
0        False           -1            0  
Training configurations :


,metrics,losses,optimizers,augment_prct,trim_audio,reduce_noise,trim_threshold,max_silence,trim_method,trim_mode,...,shuffle_size,epochs,verbose,train_times,valid_times,train_size,valid_size,test_size,pred_step,dataset_infos
0,"{'name': 'metric_list', 'dtype': 'float32', 'm...","{'tts_model_loss': {'reduction': 'none', 'name...","{'tts_model_optimizer': {'name': 'Adam', 'lear...",0.25,False,False,0.075,0.25,window,start_end,...,512,5,1,1,1,None,None,4,-1,"{'train': {'text': {'# uniques': 640}, 'filena..."


## Dataset analysis

Dataset generation is quite fast compared to the training time : approximately 15-20 batches / sec. It demonstrates that the data generation pipeline is clearly not the bottleneck of the training process. Nevertheless, you can speed up training by increasing the number of frames generated at each inference step (1 by default) (this feature has not been properly tested, some errors may occur). 

In [5]:
config = model.get_dataset_config(batch_size = 32, is_validation = False, shuffle_size = 0)
ds = prepare_dataset(dataset, ** config)

test_dataset_time(ds)

99it [00:06, 14.59it/s]


100 batchs in 6.795 sec sec (14.717 batch / sec)

Batch infos : 
Item 0 : 
 Item 0 : shape : (32, 82) - type : int32- min : 0.000 - max : 68.000
 Item 1 : shape : (32, 441, 80) - type : float32- min : -16.130 - max : 3.896
 Item 2 : shape : (32,) - type : int32- min : 153.000 - max : 441.000
Item 1 : 
 Item 0 : shape : (32, 441, 80) - type : float32- min : -11.513 - max : 1.925
 Item 1 : shape : (32, 441) - type : float32- min : 0.000 - max : 1.000


6.794913053512573

## Waveglow inference

In [2]:
#waveglow = PtWaveGlow() # for pytorch-based inference
waveglow = WaveGlow()
model    = Tacotron2(nom = model_name)

dataset  = get_dataset('siwis')

Model restoration...
Initializing submodel : `vocoder` !
Successfully restored vocoder from pretrained_models/WaveGlow/saving/vocoder.json !
[WARNING] Some layers have not bene restored from the checkpoint ! Run `model.load_checkpoint().assert_consumed()` to check if it is a critical error or not
Model WaveGlow initialized successfully !
Model restoration...
Initializing submodel : `tts_model` !
Successfully restored tts_model from pretrained_models/tacotron2_siwis/saving/tts_model.json !
Model tacotron2_siwis initialized successfully !
Loading dataset siwis...


### Inference based on dataset

This cell displays 3 different audios : 
- The original one (i.e. the human speaking)
- An *inverted* version : the prediction of `WaveGlow` on the ground truth `mel-spectrogram`
- The `Tacotron-2` generated audio

It is important to note that the general audio quality of the generated audio will (in theory) not be better than the inverted version. It therefore means that, if the inverted version is of poor quality, you will require another vocoder model for this speaker.

In [ ]:
for idx, row in dataset.sample(2, random_state = 0).iterrows():
    text     = row['text']
    filename = row['filename']
    
    encoded  = model.encode_text(text)
    
    # text analysis
    print("Text :\n  Original text : {}\n  Encoded text : {}\n  Decoded text : {}".format(
        text, encoded, model.decode_text(encoded)
    ))
    
    # mel analysis
    original_mel     = load_mel(filename, model.mel_fn)
    processed_mel    = model.get_mel_input(filename)
    
    plot_spectrogram(original = original_mel, processed = processed_mel, ncols = 2)
    
    # audio analysis
    original_audio  = load_audio(filename, rate = rate)
    inverted_audio  = waveglow.infer(original_mel)
    
    print("Original audio :")
    display_audio(original_audio, rate = rate)
    print("Waveglow inversion based on the original mel-spectrogram")
    display_audio(inverted_audio, rate = rate)
    
    # Uncomment these lines to perform Tacotron-2 inference
    _, predicted_mel, _, _ = model.infer(encoded)
    predicted_audio = waveglow.infer(predicted_mel)
    print('Predicted audio :')
    display_audio(predicted_audio, rate = rate)



### Inference based on custom text

The 1st cell uses the `tts API` while the 2nd shows the complete pipeline.

In [ ]:
from models.tts import tts

text = "Bonjour à tous ! Voici une démonstration du modèle en français."

_ = tts(text, model = model, directory = None, save = False, display = True)

In [ ]:
text = "Bonjour à tous ! Voici une démonstration du modèle en français."
# Encode text
encoded = model.encode_text(text)
# Generate mel and attention results
# The 1st output is the mel, 2nd is mel_postnet (final postnet after final processing)
# the 3rd are 'gates' (deciding when to stop generation) and 4th are attention weights
_, mel, _, attn = model.infer(encoded)
# Make inference mel --> audio
audio = waveglow.infer(mel)
# Plot spectrogram + attention
plot_spectrogram(spectrogram = mel, attention_weights = attn)
# Display audio
_ = display_audio(audio, rate = rate)